In [7]:
import glob
import os
import re
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import spacy
import textacy

In [2]:
test = os.path.join("plain", "36105213330389.txt")

In [4]:
!python3 -m spacy download en

    100% |████████████████████████████████| 37.4MB 14.9MB/s a 0:00:011
  Running setup.py install for en-core-web-sm ... done

    Linking successful
    /home/csb5t/.local/share/virtualenvs/jarndyce-XiJYJsrW/lib/python3.6/site-packages/en_core_web_sm
    -->
    /home/csb5t/.local/share/virtualenvs/jarndyce-XiJYJsrW/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [6]:
nlp = spacy.load('en')

In [34]:
nlplg = spacy.load('en_core_web_lg', disable=['parser', 'tagger', 'textact'])

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/csb5t/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [35]:
def strip_names_and_numbers(text):
    if len(text) < 1000000:
        doc = nlplg(text)
        tokens = [ent.text for ent in doc.ents if ent.label_ not in ["PERSON", "CARDINAL"]]
        return " ".join(tokens)
    if len(text) >= 1000000:
        doc = textacy.spacier.utils.make_doc_from_text_chunks(text, nlplg)
        tokens = [ent.text for ent in doc.ents if ent.label_ not in ["PERSON", "CARDINAL"]]
        return " ".join(tokens)

In [36]:
with open(test, 'r') as f:
    text = f.read()
    de_personed = strip_names_and_numbers(text)
    print(de_personed)

The White Hat &C. &C. '©• '' ' ' ' ARROWSMITH MEN IN A BOAT FRENCHMAN AMERICA Bristol Quay Street London HAMILTON, Kent & Co. Limited.*A Cracker BRISTOL Quay Street LONDON Simpkin Kent & Co. Hack” Derby Dramatic News HOUSE THE BLACK SHEEP OF THE FAMILY LAST 279 WHITE HAT.® the moon one night the next morning Regent Street THE WHITE HAT first half a minute two minutes customers’ THF a couple of minutes Downshire Grace Downshire Downshires Newmarket next week Downshire Downshire Downshire Downshire Two days later Goodwood Ramsgate Snob ’Arry ’Arry at least half an hour Two days later some days the Queen’s Hotel the afternoon morning second THE WHITE HAT Earth nearly eight o’clock Goodwood each day HACK RELATED BY U C H first 5 years old HACK first Swell HACK England Shropshire the very first day HACK yesterday HACK England eight years old Gloucestershire the Blackmoor Vale England Roman Catholics Swell ’oss one day HACK one day 172 noon HACK first before very many days Tattersall’s days 

In [37]:
def lowercase(textlist):
    return [word.lower() for word in textlist]

In [38]:
def remove_numbers(text):
    cleaned = re.sub(r'\d+', '', text)
    return cleaned

In [39]:
def remove_non_alpha_nltk(text):
    words = word_tokenize(text)
    lowered = lowercase(words)
    return " ".join([word for word in lowered if word.isalpha()])

In [40]:
def remove_single_letter_words(text):
    cleaned = ' '.join([word for word in text.split() if len(word) > 1])
    return cleaned

In [41]:
def clean_text(text):
    de_person_number = strip_names_and_numbers(text)
    fewer_nums = remove_numbers(de_person_number)
    alpha = remove_non_alpha_nltk(fewer_nums)
    cleaned = remove_single_letter_words(alpha)
    return cleaned

In [42]:
with open(test, 'r') as f:
    text = f.read()
    cleaned = clean_text(text)
    print(cleaned)

the white hat arrowsmith men in boat frenchman america bristol quay street london hamilton kent co limited cracker bristol quay street london simpkin kent hack derby dramatic news house the black sheep of the family last white the moon one night the next morning regent street the white hat first half minute two minutes customers thf couple of minutes downshire grace downshire downshires newmarket next week downshire downshire downshire downshire two days later goodwood ramsgate snob arry arry at least half an hour two days later some days the queen hotel the afternoon morning second the white hat earth nearly eight clock goodwood each day hack related by first years old hack first swell hack england shropshire the very first day hack yesterday hack england eight years old gloucestershire the blackmoor vale england roman catholics swell oss one day hack one day noon hack first before very many days tattersall days park newmarket sun day afternoon hace cambridgeshire week morrow park new

In [43]:
# Given a filename, extract just the book id
def get_book_id(fn):
    return os.path.split(fn)[1].split('_')[0].split('.')[0]

In [20]:
!mkdir tm_texts

In [21]:
!ls

LICENSE  Pipfile.lock	     parse.ipynb  textacy-analysis.ipynb  tm_texts
Pipfile  clean_for_tm.ipynb  plain	  tm-gensim.ipynb


In [58]:
def write_clean_text(book_id):
    with open(os.path.join('plain', book_id + '.txt'), 'r', encoding='latin-1') as f:
        with open(os.path.join('tm_texts', book_id + '.txt'), 'w') as g:
            text = f.read()
            g.write(clean_text(text))

In [53]:
book_ids = {get_book_id(fn) for fn in glob.glob(os.path.join('plain', '*.txt'))}

In [54]:
len(book_ids)

1417

In [47]:
import multiprocessing as mp

In [59]:
pool = mp.Pool()

In [49]:
mp.cpu_count()

24

In [60]:
%time pool.map(write_clean_text, book_ids)

CPU times: user 469 ms, sys: 391 ms, total: 859 ms
Wall time: 33min 15s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [61]:
pool.close()

In [69]:
for fn in tqdm(glob.glob(os.path.join('plain', '*.txt'))):
    book_id = get_book_id(fn)
    with open(fn, 'r') as f:
        with open(os.path.join('tm_texts', book_id + '.txt'), 'w') as g:
            text = f.read()
            g.write(clean_text(text))

100%|██████████| 1417/1417 [15:02<00:00,  1.22it/s]
